In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
import re
from bs4 import BeautifulSoup

import nltk      #add
nltk.download("stopwords")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


wv = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/Data/word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)
wv.init_sims(replace=True) 


from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import gmean

def word_averaging(wv, words):
    all_words, mean = set(), []
    cou=0
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    #mean = gensim.matutils.unitvec(gmean(np.array(mean))).astype(np.float32)
    #mean = gensim.matutils.unitvec(np.percentile(np.array(mean),25,axis=0)).astype(np.float32)
    #mean = gensim.matutils.unitvec(skew(np.array(mean))).astype(np.float32)
   #mean = gensim.matutils.unitvec(np.percentile(np.array(mean),50,axis=0)).astype(np.float32)
    return mean


def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            tokens.append(word)
    return tokens

In [6]:
nltk.download('punkt') #add
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fake.csv',encoding='latin-1')
df = df[df['text'].notna()] #add
df['text'] = df['text'].apply(clean_text)
print(df.head(10))

comtdata=df

test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values

X_comtdata_average1 = word_averaging_list(wv,test_tokenized)
np.savetxt('/content/drive/MyDrive/Colab Notebooks/Data/word2vec/w2vdata.csv',X_comtdata_average1, delimiter=',', fmt='%f')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,214,215,216,217,218,219,220,221,222,223,224,225,227,228,229,230,232,233,234,235,237,238,239,240,241,242,244,245,246,247,248,249,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,2

                                       uuid  ... Unnamed: 698
0  6a175f46bcd24d39b3e962ad0f29936721db70db  ...          NaN
1  2bdc29d12605ef9cf3f09f9875040a7113be5d5b  ...          NaN
2  c70e149fdd53de5e61c29281100b9de0ed268bc3  ...          NaN
3  7cf7c15731ac2a116dd7f629bd57ea468ed70284  ...          NaN
4  0206b54719c7e241ffe0ad4315b808290dbe6c0f  ...          NaN
5  8f30f5ea14c9d5914a9fe4f55ab2581772af4c31  ...          NaN
6  d3cc0fe38f41a59f7c48f8c3528ca5f74193148f  ...          NaN
7  b4bbf8b5c19e8864f5257832a58b81ef4ed2d4e4  ...          NaN
8  a19aabaa5a61eb8bc22fadaaa003e5fbba5c4bf6  ...          NaN
9  f54d8e13010d0a79893995ee65360ad4b38b5a35  ...          NaN

[10 rows x 699 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
